In [1]:
import Bio.PDB as bpdb
import pylab
import numpy as np
import pandas as pd
import scipy.sparse

In [15]:
kinetic_data = pd.read_pickle('kinetic_data.pkl')
if not 'RCO' in kinetic_data: kinetic_data.insert(8, "RCO", 0, False)
if not 'ACO' in kinetic_data: kinetic_data.insert(9, "ACO", 0, False)
if not 'LRO' in kinetic_data: kinetic_data.insert(10, "LRO", 0, False)
if not 'SRO' in kinetic_data: kinetic_data.insert(11, "SRO", 0, False)
if not 'LocalCO' in kinetic_data: kinetic_data.insert(12, "LocalCO", 0, False)
if not 'NonLocalCO' in kinetic_data: kinetic_data.insert(13, "NonLocalCO", 0, False)
if not 'TCD' in kinetic_data: kinetic_data.insert(14, "TCD", 0, False)
if not 'CC' in kinetic_data: kinetic_data.insert(15, "CC", 0, False)
if not 'N_alpha' in kinetic_data: kinetic_data.insert(16, "N_alpha", 0, False)

"""kinetic_data = kinetic_data[
    kinetic_data['pdb'].isin([
        '1LMB(Chain 3)', '1UBQ', '2PTL(18-77)', '1APS',
        '1CSP', '1TEN', '1SHF(Chain A)', '1PGB', '1PGB(41-56)',
        '1PIN(6-39)', '2PDD', '1IMQ', '1FNF', '1WIT', '1SHG',
        '1SRL', '1PNJ', '1G6P', '1MJC', '1LOP(Chain A)', '1FKB',
        '2CI2', '1URN(Chain A)'])
]#"""
kinetic_data

,name,pdb,kinetic_state,class,pdb_length,midpoint,log_kf,log_ku,RCO,ACO,LRO,SRO,LocalCO,NonLocalCO,TCD,CC,N_alpha,deltaG,source
0,Colicin E7 immunity protein,1AYI,Two,α,85,1.22,3.13,1.00,0.078630,6.762223,3.279070,5.104651,3.248292,36.901695,0.790157,0,0,-2.90,"Maxwell KL, Wildes D, Zarrine-Afsar A, De Los ..."
1,"Telomeric protein DNA-binding domain, human",1BA5,Two,α,49,0.69,2.56,0.52,0.085722,4.543284,2.471698,4.924528,3.210728,25.235714,0.646137,0,0,-2.78,"Garbuzynskiy SO, Ivankov DN, Bogatyreva NS, Fi..."
2,Immunoglobulin binding B-domain,1BDD(2-59),Two,α,58,2.52,5.08,1.82,0.070733,4.102534,2.827586,5.103448,3.266892,27.704142,0.704221,0,0,-4.44,"Garbuzynskiy SO, Ivankov DN, Bogatyreva NS, Fi..."
3,16th domain of brain α-spectrin,1CUN(7-112),Two,α,106,-0.87,2.08,-2.61,0.080790,8.563726,3.462264,5.603774,3.402357,54.536388,0.741812,0,0,-6.40,"Garbuzynskiy SO, Ivankov DN, Bogatyreva NS, Fi..."
4,17th domain of brain α-spectrin,1CUN(113-219),Two,α,107,-1.48,1.48,-3.39,0.080594,8.623584,3.504673,5.616822,3.402662,54.497354,0.770198,0,0,-6.64,"Garbuzynskiy SO, Ivankov DN, Bogatyreva NS, Fi..."
5,"FADD death-domain, human",1E41(93-192),Two,α,100,-0.26,2.95,-1.30,0.059760,5.975976,3.570000,5.400000,3.331481,42.772973,0.770600,0,0,-5.81,"Garbuzynskiy SO, Ivankov DN, Bogatyreva NS, Fi..."
6,"Rap1 myb-domain, human",1FEX,Two,α,59,1.69,3.56,1.26,0.080263,4.735522,2.694915,5.067797,3.260870,30.060976,0.751221,0,0,-3.14,"Garbuzynskiy SO, Ivankov DN, Bogatyreva NS, Fi..."
7,Myb transforming protein,1IDY,Two,α,54,1.35,3.78,0.74,0.076444,4.127954,2.111111,5.074074,3.244526,25.899160,0.606310,0,0,-4.15,"Garbuzynskiy SO, Ivankov DN, Bogatyreva NS, Fi..."
8,Colicin E9 immunity protein,1IMQ,Two,α,85,-0.61,3.17,-0.83,0.095965,8.252947,3.976744,5.058140,3.239080,37.036827,0.934965,0,0,-5.45,"Maxwell KL, Wildes D, Zarrine-Afsar A, De Los ..."
9,Trp-Cage Miniprotein,1L2Y,Two,α,20,5.65,5.43,4.99,0.139645,2.792897,0.550000,4.400000,3.147727,15.153846,0.652500,0,0,-0.59,"Garbuzynskiy SO, Ivankov DN, Bogatyreva NS, Fi..."


In [3]:
def calc_residue_dist(residue_one, residue_two) :
    """Returns the C-alpha distance between two residues"""
    diff_vector  = residue_one["CA"].coord - residue_two["CA"].coord
    return np.sqrt(np.sum(diff_vector * diff_vector))

def calc_min_dist(r1, r2):
    m = np.inf
    for a1 in r1.get_atoms():
        if a1.name == 'H': continue
        for a2 in r2.get_atoms():
            if a2.name == 'H': continue
            v = a1.coord - a2.coord
            d = np.sqrt(np.sum(v*v))
            m = min(d, m)
    return m
            

def calc_dist_matrix(chain_one, chain_two, distance_metric) :
    """Returns a matrix of C-alpha distances between two chains"""
    answer = np.zeros((len(chain_one), len(chain_two)), float)
    for row, residue_one in enumerate(chain_one) :
        for col, residue_two in enumerate(chain_two) :
            answer[row, col] = distance_metric(residue_one, residue_two)
    return answer

def calc_atom_dist_matrix(r1, r2):
    matrix = np.zeros((len(list(r1.get_atoms())), len(list(r2.get_atoms()))))
    for i, a1 in enumerate(r1.get_atoms()):
        for j, a2 in enumerate(r2.get_atoms()):
            if a1.name == 'H' or a2.name == 'H': matrix[i,j] = np.nan
            v = a1.coord - a2.coord
            matrix[i,j] = np.sqrt(np.sum(v*v))
    return matrix

In [4]:
R_cutoff = 6.0

'''
ca_dist_matricies = {}
dist_matricies = {}
models = {}

for index, entry in kinetic_data.iterrows():
    struct = bpdb.PDBParser().get_structure(entry.pdb[:4], f'pdb_files/{entry.pdb}.pdb')
    model = struct[0]

    dist_matrix = calc_dist_matrix(list(struct.get_residues()), list(struct.get_residues()), calc_min_dist)
    dist_matricies[entry.pdb] = dist_matrix

    ca_dist_matrix = calc_dist_matrix(list(struct.get_residues()), list(struct.get_residues()), calc_residue_dist)
    ca_dist_matricies[entry.pdb] = ca_dist_matrix
    models[entry.pdb] = model
#'''

/home/sabsr3/.local/lib/python3.8/site-packages/Bio/PDB/PDBParser.py:395: PDBConstructionWarning: Ignoring unrecognized record 'END' at line 697
  warnings.warn(
/home/sabsr3/.local/lib/python3.8/site-packages/Bio/PDB/PDBParser.py:395: PDBConstructionWarning: Ignoring unrecognized record 'END' at line 473
  warnings.warn(
/home/sabsr3/.local/lib/python3.8/site-packages/Bio/PDB/PDBParser.py:395: PDBConstructionWarning: Ignoring unrecognized record 'END' at line 469
  warnings.warn(
/home/sabsr3/.local/lib/python3.8/site-packages/Bio/PDB/PDBParser.py:395: PDBConstructionWarning: Ignoring unrecognized record 'END' at line 872
  warnings.warn(
/home/sabsr3/.local/lib/python3.8/site-packages/Bio/PDB/PDBParser.py:395: PDBConstructionWarning: Ignoring unrecognized record 'END' at line 847
  warnings.warn(
/home/sabsr3/.local/lib/python3.8/site-packages/Bio/PDB/PDBParser.py:395: PDBConstructionWarning: Ignoring unrecognized record 'END' at line 804
  warnings.warn(
/home/sabsr3/.local/lib/pyth

## Calculate RCO and ACO

In [5]:
R_cutoff = 6.0

for index, entry in kinetic_data.iterrows():
    model = models[entry.pdb]
    dist_matrix = dist_matricies[entry.pdb]

    RCO_tmp = 0 # Non-Normalized RCO
    N = 0
    for i, r1 in enumerate(list(model.get_residues())):
        for j, r2 in enumerate(list(model.get_residues())):
            if i>=j: continue
            if dist_matrix[i][j] > R_cutoff: continue
            atom_dist_matrix = calc_atom_dist_matrix(r1, r2)

            n = np.sum(np.triu(atom_dist_matrix <= R_cutoff))
            N += n
            RCO_tmp += abs(j-i) * n
    RCO = RCO_tmp/(N*dist_matrix.shape[0])
    ACO = RCO_tmp/N
    kinetic_data.loc[index, 'RCO'] = RCO
    kinetic_data.loc[index, 'ACO'] = ACO

## Calculate LRO

In [6]:
R_cutoff = 8.0

for index, entry in kinetic_data.iterrows():
    model = models[entry.pdb]
    dist_matrix = ca_dist_matricies[entry.pdb]

    LRO_tmp = 0 # Non-Normalized RCO
    N = 0
    for i in range(dist_matrix.shape[0]):
        for j in range(i+1, dist_matrix.shape[1]):
            if dist_matrix[i][j] > R_cutoff: continue
            N += 1
            LRO_tmp += 1 if abs(j-i) > 12 else 0
    LRO = LRO_tmp / dist_matrix.shape[0]
    kinetic_data.loc[index, 'LRO'] = LRO

## Calculate SRO

In [7]:
R_cutoff = 8.0
Smax = 6

for index, entry in kinetic_data.iterrows():
    model = models[entry.pdb]
    dist_matrix = dist_matricies[entry.pdb]

    SRO_tmp = 0 # Non-Normalized RCO
    N = 0
    for i, r1 in enumerate(list(model.get_residues())):
        for j, r2 in enumerate(list(model.get_residues())):
            if i>=j: continue
            if dist_matrix[i][j] > R_cutoff: continue
            if abs(i-j) > Smax: continue
            atom_dist_matrix = calc_atom_dist_matrix(r1, r2)

            n = np.sum(np.triu(atom_dist_matrix <= R_cutoff))
            
            N += n
            SRO_tmp += n
    SRO = SRO_tmp / dist_matrix.shape[0]
    kinetic_data.loc[index, 'SRO'] = SRO

## Calculate LocalCO

In [8]:
R_cutoff = 8.0
Smax = 6

for index, entry in kinetic_data.iterrows():
    model = models[entry.pdb]
    dist_matrix = dist_matricies[entry.pdb]
    
    LCO_temp = 0 # Non-Normalized RCO
    N = 0
    for i, r1 in enumerate(list(model.get_residues())):
        for j, r2 in enumerate(list(model.get_residues())):
            if i>=j: continue
            if dist_matrix[i][j] > R_cutoff: continue
            if abs(i-j) > Smax: continue
            atom_dist_matrix = calc_atom_dist_matrix(r1, r2)

            n = np.sum(np.triu(atom_dist_matrix <= R_cutoff))

            N += n
            LCO_temp += abs(j-i) * n
    LCO = LCO_temp / N
    kinetic_data.loc[index, 'LocalCO'] = LCO

## Calculate NonLocal Contact Order

In [9]:
R_cutoff = 8.0
Smin = 12

for index, entry in kinetic_data.iterrows():
    model = models[entry.pdb]
    dist_matrix = dist_matricies[entry.pdb]

    NLCO_temp = 0 # Non-Normalized RCO
    N = 0
    for i, r1 in enumerate(list(model.get_residues())):
        for j, r2 in enumerate(list(model.get_residues())):
            if i>=j: continue
            if dist_matrix[i][j] > R_cutoff: continue
            if abs(i-j) < Smin: continue
            atom_dist_matrix = calc_atom_dist_matrix(r1, r2)

            n = np.sum(np.triu(atom_dist_matrix <= R_cutoff))

            N += n
            NLCO_temp += abs(j-i) * n
    NLCO = NLCO_temp / N
    kinetic_data.loc[index, 'NonLocalCO'] = NLCO

## Calculate Total Contact Distance

In [10]:
R_cutoff = 6.0
lcut = 2

for index, entry in kinetic_data.iterrows():
    model = models[entry.pdb]
    dist_matrix = dist_matricies[entry.pdb]

    TCD_temp = 0 # Non-Normalized RCO
    N = 0
    for i, r1 in enumerate(list(model.get_residues())):
        for j, r2 in enumerate(list(model.get_residues())):
            if i>=j: continue
            if dist_matrix[i][j] > R_cutoff: continue
            if abs(j-i) <= lcut: continue

            atom_dist_matrix = calc_atom_dist_matrix(r1, r2)

            n = np.sum(np.triu(atom_dist_matrix <= R_cutoff))
            
            N += 1
            TCD_temp += abs(j-i)
    TCD = TCD_temp / (dist_matrix.shape[0] * dist_matrix.shape[0])
    kinetic_data.loc[index, 'TCD'] = TCD

## Clustering Coefficient

In [11]:
R_cutoff = 4.5

for index, entry in kinetic_data.iterrows():
    model = models[entry.pdb]
    dist_matrix = dist_matricies[entry.pdb]

    CC_temp = np.zeros(dist_matrix.shape[0]) # Non-Normalized CC
    for i, r1 in enumerate(list(model.get_residues())):
        Nc = 0
        for j, r2 in enumerate(list(model.get_residues())):
            if i==j: continue
            if abs(i-j) > 1 and dist_matrix[i][j] > R_cutoff: continue
            if dist_matrix[i][j] <= R_cutoff: Nc += 1
            for k, r3 in enumerate(list(model.get_residues())):
                if j >= k: continue
                if i==k: continue
                if dist_matrix[j][k] > R_cutoff: continue
                if abs(i-k) > 1 and dist_matrix[i][k] > R_cutoff: continue

                CC_temp[i] += 1
        #print(f'{CC_temp[i]}, {Nc}')
        if (CC_temp[i] > 0):
            CC_temp[i] /= Nc*(Nc-1)/2
    CC = np.mean(CC_temp)
    kinetic_data.loc[index, 'CC'] = CC

## Geometric Contact Number

In [12]:
from scipy.spatial import Delaunay
from scipy.spatial import Voronoi
from scipy.spatial import ConvexHull
import shapely
R_cutoff = 6.5

for index, entry in kinetic_data.iterrows():
    model = models[entry.pdb]
    dist_matrix = ca_dist_matricies[entry.pdb]
    residues = list(model.get_residues())

    points = np.zeros((len(residues), 3))
    for i, residue in enumerate(residues):
        for a in residue.get_atoms():
            if a.name == 'CA': points[i] = a.coord
    tri = Delaunay(points, qhull_options='QJ')
    
    contacts = set()
    for simplex in tri.simplices:
        for i in range(4):
            for j in range(i+1, 4):
                r1 = simplex[i]; r2 = simplex[j]
                if dist_matrix[r1][r2] > R_cutoff or abs(r1-r2) < 5: continue
                contacts.add(r1); contacts.add(r2)
    #kinetic_data.loc[index, 'N_alpha'] = len(contacts)
    

In [13]:
pd.set_option('display.max_rows', None)
#kinetic_data[['pdb', 'pdb_length', 'RCO', 'ACO', 'LRO', 'SRO', 'LocalCO', 'NonLocalCO', 'TCD', 'CC', 'log_kf', 'log_ku', 'deltaG']].sort_values('pdb')
kinetic_data[['pdb', 'pdb_length', 'N_alpha', 'log_kf', 'log_ku', 'deltaG']].sort_values('pdb')

,pdb,pdb_length,N_alpha,log_kf,log_ku,deltaG
90,1AON(191-345),155,0,-0.65,-2.48,-2.49
50,1APS,98,0,-0.69,-3.91,-4.38
73,1AU7(103-160),58,0,4.21,2.08,-2.90
74,1AUE,92,0,2.61,-2.35,-6.75
0,1AYI,85,0,3.13,1.00,-2.90
1,1BA5,49,0,2.56,0.52,-2.78
75,1BD8,156,0,1.26,-1.00,-3.08
2,1BDD(2-59),58,0,5.08,1.82,-4.44
91,1BFE,110,0,1.30,-1.48,-3.79
92,1BTA,89,0,1.48,-1.17,-3.61


In [14]:
kinetic_data.to_pickle('kinetic_data_co.pkl')